In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

In [ ]:
path_data = "data/" 

# Leitura dos CSVs
orders = pd.read_csv(path_data + 'olist_orders_dataset.csv')
items = pd.read_csv(path_data + 'olist_order_items_dataset.csv')
customers = pd.read_csv(path_data + 'olist_customers_dataset.csv')
sellers = pd.read_csv(path_data + 'olist_sellers_dataset.csv')
products = pd.read_csv(path_data + 'olist_products_dataset.csv')
geolocation = pd.read_csv(path_data + 'olist_geolocation_dataset.csv')

print("Datasets Olist carregados com sucesso.")

In [ ]:
# 1. Agrupar Geo (Centróide do CEP)
geo_ref = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# 2. Unificar Tabelas
items_first = items.drop_duplicates('order_id', keep='first')
df = orders.merge(items_first, on='order_id', how='left')
df = df.merge(customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='left')
df = df.merge(sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id', how='left')
df = df.merge(products[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']], on='product_id', how='left')

# 3. Merge das Coordenadas
# Destino (Cliente)
df = df.merge(geo_ref, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left').rename(columns={'geolocation_lat': 'lat_cust', 'geolocation_lng': 'lng_cust'}).drop('geolocation_zip_code_prefix', axis=1)

# Origem (Vendedor)
df = df.merge(geo_ref, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left').rename(columns={'geolocation_lat': 'lat_sell', 'geolocation_lng': 'lng_sell'}).drop('geolocation_zip_code_prefix', axis=1)

# 4. Limpeza e Target
df = df[df['order_status'] == 'delivered'].dropna(subset=['order_delivered_customer_date', 'lat_cust', 'lat_sell', 'product_weight_g'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Target: Dias Reais
df['dias_reais'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
df = df[(df['dias_reais'] > 0) & (df['dias_reais'] < 60)]

# 5. Feature: Distância (Haversine)
def haversine_vec(lat1, lon1, lat2, lon2):
    """Calcula a distância Haversine (em Km) entre dois pares de coordenadas."""
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

df['distancia_km'] = haversine_vec(df['lat_sell'], df['lng_sell'], df['lat_cust'], df['lng_cust'])
df['volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

features = ['distancia_km', 'product_weight_g', 'volume_cm3', 'freight_value', 'price']
X = df[features]
y = df['dias_reais']

print(f"Pré-processamento concluído. {len(df)} registros válidos para treinamento.")

In [ ]:
print(f"Iniciando treinamento do modelo Random Forest...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Avaliação Rápida
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f"Treinamento finalizado. Erro Absoluto Médio (MAE): {mae:.2f} dias.")

In [ ]:
print("Gerando arquivo de comparação...")

# 1. Calcular a Estimativa Antiga (Necessário para o comparativo)
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['dias_estimados_antigo'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.days

# 2. Calcular a nova previsão do modelo (dias_previstos_ia)
df['dias_previstos_ia'] = model.predict(X)

# 3. Salvar uma amostra de 5000 linhas
df_final = df[['dias_reais', 'dias_estimados_antigo', 'dias_previstos_ia']].sample(min(5000, len(df)), random_state=42)
df_final.to_csv(path_data + 'comparativo_modelo.csv', index=False)

print(f"Arquivo 'comparativo_modelo.csv' gerado em {path_data}")

In [ ]:
# Salvar o modelo treinado (para uso futuro)
joblib.dump(model, 'models/modelo_entregas.joblib')

# Salvar a referência de geolocalização (para evitar processar novamente)
geo_ref.to_csv(path_data + 'referencia_geo.csv', index=False) 

print("Modelo e dados de referência salvos localmente.")